# <div style="background-color: #b74ff8ff; padding:10px; border-radius:10px; color: #ffffffff; font-weight: bold"> 📝 Notebook de ejercicios </div>

<div style="background-color: #acf0f7ff; padding:10px; border-radius:10px"> Este notebook esta destinado a exponer

1. **Ejercicio propuesto**
2. **Desafio corto**

Ambos propuestos por el equipo **Inteligencia Artesanal**🧉</div>
<br>

---

## <div style="background-color: #ff83b7ff; padding:10px; border-radius:10px; color: #ffffffff"> 1. Ejercicio Propuesto </div>

<div style="background-color: #abdbe3ff; padding:10px; border-radius:10px"> <p>Imagina que eres analista de datos en una empresa que recibe registros de clientes desde múltiples fuentes (formularios web, exportes legacy y hojas de cálculo). <em>Los datos llegan con errores típicos: marcadores de faltantes inconsistentes ("N/A", "NULL", "?", cadena vacía), símbolos en números ($, ,), fechas en formatos mixtos y texto con espacios/uso de mayúsculas irregular. Y debes organizar estos datos para presentarlos a los directivos. <strong>¿Cómo lo harías?</strong></em></p></div>

<div><p><strong>Para hacerlo te damos unas pistas 💡:</strong></p></div>
<div> 
    <ol style="list-style:none; padding-left:0;">
    <li style="margin:6px 0;">
        <span style="display:inline-block; width:30px; height:24px; line-height:24px; text-align:center; border-radius:6px; background:#ef4444; color:#fff; font-weight:700; margin-right:10px;">1</span>
        Detectar valores nulos como <code>N/A</code>, <code>NULL</code>, <code>?</code> y convertirlos a <code>NaN</code>.
    </li>
    <li style="margin:6px 0;">
        <span style="display:inline-block; width:30px; height:24px; line-height:24px; text-align:center; border-radius:6px; background:#f59e0b; color:#fff; font-weight:700; margin-right:10px;">2</span>
        Reemplazar marcadores faltantes por <code>NaN</code>.
    </li>
    <li style="margin:6px 0;">
        <span style="display:inline-block; width:30px; height:24px; line-height:24px; text-align:center; border-radius:6px; background:#10b981; color:#fff; font-weight:700; margin-right:10px;">3</span>
        Convertir tipos de datos: números y fechas.
    </li>
    <li style="margin:6px 0;">
        <span style="display:inline-block; width:30px; height:24px; line-height:24px; text-align:center; border-radius:6px; background:#60a5fa; color:#fff; font-weight:700; margin-right:10px;">4</span>
        Limpiar texto como <em>Customer Name</em> y <em>Gender</em>.
    </li>
    </ol>
</div>

>🚨Recuerda🚨 <em>Para realizar este ejercicio puedes encontrar toda la información teorica necesaria en <code>Notebook_demo</code> </em>
>
> Para poder hacer este ejercicio vamos a usar un Dataset en crudo.

### <div style="color: #dda639ff">Dataset en crudo antes de empezar </div>

In [9]:
import pandas as pd
import numpy as np
import os
import sqlite3
from IPython.display import display

# Leer JSON directamente en un DataFrame
df = pd.read_json("datasets/clientes.json", encoding="utf-8")

print("📊 Dataset de clientes cargado desde JSON:")
print(f"   📄 Dimensiones: {df.shape[0]} filas x {df.shape[1]} columnas")

print("\n🔍 Primeras 5 filas del dataset crudo:")
display(df.head(6))

# Guardar copia del dataset original para comparación posterior
df_original = df.copy()


📊 Dataset de clientes cargado desde JSON:
   📄 Dimensiones: 6 filas x 4 columnas

🔍 Primeras 5 filas del dataset crudo:


,Customer Name,Income,Signup Date,Gender
0,John Doe,"$50,000",12/05/2022,M
1,JANE DOE,N/A,2021-07-30,Female
2,Alice smith,"$2,500",15/07/2020,female
3,Mário Santos,10000,15/07/2023,F
4,,?,N/A,?
5,NULL,NULL,NULL,NULL


### <div style="background-color: #fbc7ddff; padding:10px; border-radius:10px"> 🚀 Solución al ejercicio propuesto </div>
#### <div style="color: #dda639ff"> 1. Detectar valores nulos como `N/A`,`NULL,?` y convertirlos a `NaN`. </div>
Antes de limpiar, verificamos cuántos **nulos reales** detecta pandas (notar que aún no convertimos marcadores como `N/A`, `NULL`, `?` a `NaN`).

In [10]:
print("Valores nulos antes de la limpieza:")
df.isna().sum()

Valores nulos antes de la limpieza:


Customer Name    0
Income           0
Signup Date      0
Gender           0
dtype: int64

#### <div style="color: #dda639ff"> 2. Reemplazar marcadores faltantes por `NaN` </div>
Convertimos `""` (cadena vacía), `"?"`, `"N/A"` y `"NULL"` a `NaN` para que pandas los trate como faltantes.

In [11]:
df = df.replace(["", "?", "N/A", "NULL"], np.nan)
print("Valores nulos después del reemplazo:")
df.isna().sum()

Valores nulos después del reemplazo:


Customer Name    2
Income           3
Signup Date      2
Gender           2
dtype: int64


#### <div style="color: #dda639ff"> 3. Convertir tipos de datos: números y fechas. </div>
**Income:** remover símbolos (`$`, `,`) y convertir a numérico.  
**Signup Date:** convertir a `datetime`, tolerando formatos mixtos y errores.


In [12]:
# Income a numérico (con manejo seguro de NaN)
df['Income'] = df['Income'].astype(str).str.replace(r'[\$,]', '', regex=True)
df['Income'] = pd.to_numeric(df['Income'], errors='coerce')

# Signup Date a datetime (dd/mm/yyyy y yyyy-mm-dd)
df['Signup Date'] = pd.to_datetime(df['Signup Date'], dayfirst=True, errors='coerce')

print(" Tipos de datos después de conversión:")
df.dtypes

 Tipos de datos después de conversión:


Customer Name            object
Income                  float64
Signup Date      datetime64[ns]
Gender                   object
dtype: object


#### <div style="color: #dda639ff"> 4. Limpiar texto como `Customer Name` y `Gender` </div>
- `Customer Name`: quitar espacios extra y normalizar a *Title Case*.  
- `Gender`: normalizar a categorías **Male/Female**.


In [13]:
# Customer Name: strip + espacios múltiples a uno + Title Case (manejo seguro)
df['Customer Name'] = df['Customer Name'].astype(str).str.strip()
df['Customer Name'] = df['Customer Name'].str.replace(r'\s+', ' ', regex=True)
df['Customer Name'] = df['Customer Name'].str.title()

# Gender: normalizar (manejo seguro de NaN)
df['Gender'] = df['Gender'].astype(str).str.strip().str.lower()
df['Gender'] = df['Gender'].replace({'m': 'male', 'male': 'male', 'f': 'female', 'female': 'female', 'nan': np.nan})
df['Gender'] = df['Gender'].str.capitalize()

print("Limpieza de texto completada:")
df[['Customer Name','Gender']]

Limpieza de texto completada:


,Customer Name,Gender
0,John Doe,Male
1,Jane Doe,Female
2,Alice Smith,Female
3,Mário Santos,Female
4,Nan,NaN
5,Nan,NaN


Una vez que tenemos el dataset "limpio" podemos hacer lo siguiente:
- (Opcional) Remover filas sin información clave.  
- Resetear el índice.  
- Revisar el estado final del DataFrame.

Esto permitira ver solo los registros con datos relevantes

In [14]:

# Ejemplo: eliminar filas sin nombre e ingreso
df_clean = df.dropna(subset=['Customer Name', 'Income'], how='any').reset_index(drop=True)

print("✅ Filas válidas después de limpieza:")
print(f"   📊 Antes: {len(df)} filas")
print(f"   📊 Después: {len(df_clean)} filas")
print(f"   🗑️ Eliminadas: {len(df) - len(df_clean)} filas")

print("\n📋 Tipos de datos finales:")
display(df_clean.dtypes)

print("\n📄 DataFrame final:")
df_clean


✅ Filas válidas después de limpieza:
   📊 Antes: 6 filas
   📊 Después: 3 filas
   🗑️ Eliminadas: 3 filas

📋 Tipos de datos finales:


Customer Name            object
Income                  float64
Signup Date      datetime64[ns]
Gender                   object
dtype: object


📄 DataFrame final:


,Customer Name,Income,Signup Date,Gender
0,John Doe,50000.0,2022-05-12,Male
1,Alice Smith,2500.0,2020-07-15,Female
2,Mário Santos,10000.0,2023-07-15,Female


#### <div style="color: #dda639ff"> Etapa final: **Antes vs. Después** </div>
Para visualizar el impacto de la limpieza, mostramos una tabla comparativa con algunas filas del dataset original frente al dataset limpio.

In [21]:

print("📊 COMPARACIÓN COMPLETA - ANTES VS DESPUÉS")
print("=" * 60)

print("\n🔴 ANTES (datos crudos):")
print("-" * 30)
display(df_original)

print("\n🟢 DESPUÉS (datos limpios):")
print("-" * 30)
display(df_clean)

# Resumen de la transformación
print(f"\n📊 RESUMEN DE LA LIMPIEZA:")
print(f"   📊 Registros originales: {len(df_original)}")
print(f"   📊 Registros finales: {len(df_clean)}")
print(f"   🗑️ Registros eliminados: {len(df_original) - len(df_clean)}")
print(f"   📈 Tasa de conservación: {(len(df_clean)/len(df_original)*100):.1f}%")

📊 COMPARACIÓN COMPLETA - ANTES VS DESPUÉS

🔴 ANTES (datos crudos):
------------------------------


,Customer Name,Income,Signup Date,Gender
0,John Doe,"$50,000",12/05/2022,M
1,JANE DOE,N/A,2021-07-30,Female
2,Alice smith,"$2,500",15/07/2020,female
3,Mário Santos,10000,15/07/2023,F
4,,?,N/A,?
5,NULL,NULL,NULL,NULL



🟢 DESPUÉS (datos limpios):
------------------------------


,Customer Name,Income,Signup Date,Gender
0,John Doe,50000.0,2022-05-12,Male
1,Alice Smith,2500.0,2020-07-15,Female
2,Mário Santos,10000.0,2023-07-15,Female



📊 RESUMEN DE LA LIMPIEZA:
   📊 Registros originales: 6
   📊 Registros finales: 3
   🗑️ Registros eliminados: 3
   📈 Tasa de conservación: 50.0%


#### <div style="color: #dda639ff"> Paso extra 📎 </div>

Como actividad extra podemos exportar el dataset resultante en diferentes formatos.

La idea es **Crear una carpeta "export" donde se alojaran los archivos con el dataset resultante.**
Podemos convertir el dataset completo en tipos de archivos que nos sean utiles, por ejemplo los que vimos en la teoria (csv, excel, json, sql)

In [14]:
# Crear carpeta "export" si no existe
os.makedirs("export", exist_ok=True)

print("Exportando dataset de clientes limpios...")

# 1) CSV - formato más universal para análisis de datos
df_clean.to_csv("export/clientes_limpios.csv", index=False, encoding="utf-8")
print("CSV exportado: export/clientes_limpios.csv")

# 2) Excel - formato empresarial estándar
df_clean.to_excel("export/clientes_limpios.xlsx", index=False)
print("Excel exportado: export/clientes_limpios.xlsx")

# 3) JSON - formato para APIs y aplicaciones web
df_clean.to_json("export/clientes_limpios.json", orient="records", indent=2, force_ascii=False)
print("JSON exportado: export/clientes_limpios.json")

# 4) SQL (SQLite) - base de datos para consultas complejas
con = sqlite3.connect("export/clientes_database.db")
df_clean.to_sql("clientes_limpios", con, if_exists="replace", index=False)
con.close()
print("Base de datos SQLite creada: export/clientes_database.db")



Exportando dataset de clientes limpios...
CSV exportado: export/clientes_limpios.csv
Excel exportado: export/clientes_limpios.xlsx
JSON exportado: export/clientes_limpios.json
Base de datos SQLite creada: export/clientes_database.db


---
## <div style="background-color: #ff83b7ff; padding:10px; border-radius:10px; color: #ffffffff"> 2. Desafio corto </div>

***Objetivo:** detectar y contar valores faltantes en un dataset pequeño.*

### 📝 Consigna
Con el CSV de ejemplo que se muestra al final, realizar las siguientes acciones:

1. Cargar el CSV
2. Mostrar el Dataframe original
3. Comprobar los valores faltantes
4. Imputar los valores de de edades faltantes con *0*
5. Imprimir el Dataframe final
6. Exportar a su extencion de preferencia
   


**CSV de ejemplo**
   ```csv
   id,nombre,edad
   1,Ana,23
   2,Beto,
   3,Caro,31
   4,,27
   5,Diego,


In [15]:
import pandas as pd
from io import StringIO

# 1) Cargar CSV
csv = StringIO("""id,nombre,edad
1,Ana,23
2,Beto,
3,Caro,31
4,,27
5,Diego,
""")
df = pd.read_csv(csv)

# 2) Mostrar DataFrame
print("DataFrame original:\n", df, "\n")

# 3) Valores faltantes
print("Valores faltantes por columna:\n", df.isna().sum(), "\n")

faltantes_pct = df["edad"].isna().mean() * 100
print(f"Porcentaje de valores faltantes en 'edad': {faltantes_pct:.2f}%\n")


DataFrame original:
    id nombre  edad
0   1    Ana  23.0
1   2   Beto   NaN
2   3   Caro  31.0
3   4    NaN  27.0
4   5  Diego   NaN 

Valores faltantes por columna:
 id        0
nombre    1
edad      2
dtype: int64 

Porcentaje de valores faltantes en 'edad': 40.00%

